# MLOPT Knapsack Example

In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd
import logging

import mlopt
from mlopt.sampling import uniform_sphere_sample
from mlopt.learners import XGBoost
from mlopt.utils import n_features, pandas2array

## Generate problem data

In [2]:
np.random.seed(1)  # Reset random seed for reproducibility

# Variable
n = 10
x = cp.Variable(n, integer=True)

# Cost
c = np.random.rand(n)

# Weights
a = cp.Parameter(n, nonneg=True, name='a')
x_u = cp.Parameter(n, nonneg=True, name='x_u')
b = 0.5 * n

## Create optimizer object

In [3]:
# Problem
cost = - c @ x
constraints = [a @ x <= b,
               0 <= x, x <= x_u]


# Define optimizer
# If you just want to remove too many messages
# change INFO to WARNING
m = mlopt.Optimizer(cp.Minimize(cost), constraints,
                    log_level=logging.INFO)

## Define training and testing parameters

In [4]:
# Average request
theta_bar = 2 * np.ones(2 * n)
radius = 1.0


def sample(theta_bar, radius, n=100):

    # Sample points from multivariate ball
    ndim = int(len(theta_bar)/2)
    X_a = uniform_sphere_sample(theta_bar[:ndim], radius, n=n)
    X_u = uniform_sphere_sample(theta_bar[ndim:], radius, n=n)

    df = pd.DataFrame({
        'a': list(X_a),
        'x_u': list(X_u)
        })

    return df


# Training and testing data
n_train = 1000
n_test = 100
theta_train = sample(theta_bar, radius, n=n_train)
theta_test = sample(theta_bar, radius, n=n_test)

## Train predictor

In [5]:
params = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [50],
}
m.train(theta_train, learner=mlopt.XGBOOST, params=params, n_folds=2)


Use new data
Compute tight constraints for training set (n_jobs = 4)


100%|██████████| 1000/1000 [00:12<00:00, 79.95it/s]


Encoding strategies
Getting unique set of strategies
Found 43 unique strategies
Caching KKT solver factors for each strategy (it works only for QP-representable problems with parameters only in constraints RHS)


Converting dataframe to array: 100%|██████████| 1000/1000 [00:00<00:00, 6154.80it/s]

Train XGBoost


...

[CV]  learning_rate=0.01, max_depth=1, n_estimators=50, objective=multi:softmax, seed=0, score=0.698, total=   1.4s
[CV] learning_rate=0.01, max_depth=5, n_estimators=50, objective=multi:softmax, seed=0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.9s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=5, n_estimators=50, objective=multi:softmax, seed=0, score=0.756, total=   2.1s
[CV] learning_rate=0.01, max_depth=5, n_estimators=50, objective=multi:softmax, seed=0 
[CV]  learning_rate=0.01, max_depth=5, n_estimators=50, objective=multi:softmax, seed=0, score=0.750, total=   2.0s
[CV] learning_rate=0.01, max_depth=10, n_estimators=50, objective=multi:softmax, seed=0 
[CV]  learning_rate=0.01, max_depth=10, n_estimators=50, objective=multi:softmax, seed=0, score=0.758, total=   2.4s
[CV] learning_rate=0.01, max_depth=10, n_estimators=50, objective=multi:softmax, seed=0 
[CV]  learning_rate=0.01, max_depth=10, n_estimators=50, objective=multi:softmax, seed=0, score=0.752, total=   3.3s
[CV] learning_rate=0.1, max_depth=1, n_estimators=50, objective=multi:softmax, seed=0 
[CV]  learning_rate=0.1, max_depth=1, n_estimators=50, objective=multi:softmax, seed=0, score=0.690, total=   1.1s
[CV] learning_rate=0.1, max_depth=1, n_estimators=50, objective=mu

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   23.5s finished


Best parameters
{'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 50, 'objective': 'multi:softmax', 'seed': 0}
Tree training time 28.42


## Benchmark on testing dataset

In [6]:
results = m.performance(theta_test)
print("Accuracy: %.2f " % results[0]['accuracy'])

Performance evaluation
Compute tight constraints for test set (n_jobs = 1)


  0%|          | 0/100 [00:00<?, ?it/s]

Academic license - for non-commercial use only


Converting dataframe to array: 100%|██████████| 100/100 [00:00<00:00, 4223.83it/s]

Predict tight constraints for test set



100%|██████████| 100/100 [00:09<00:00, 10.36it/s]

Accuracy: 88.00 


## Save training data

In [7]:
m.save_training_data("training_data.pkl", delete_existing=True)

## Create new solver and train passing loaded data

In [9]:
m = mlopt.Optimizer(cp.Minimize(cost), constraints)
m.load_training_data("training_data.pkl")
m.train(learner=mlopt.XGBOOST, params=params)  # Train after loading samples

results = m.performance(theta_test)
print("Accuracy: %.2f " % results[0]['accuracy'])

Caching KKT solver factors for each strategy (it works only for QP-representable problems with parameters only in constraints RHS)


Converting dataframe to array: 100%|██████████| 1000/1000 [00:00<00:00, 6576.13it/s]

Train XGBoost



/Users/sidereus/miniconda3/envs/python37/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0 
[CV]  batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0, score=0.700, total=   1.6s
[CV] batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0 


...

Best parameters
{'batch_size': 32, 'learning_rate': 0.1, 'max_depth': 10, 'n_epochs': 20, 'n_estimators': 50, 'n_hidden': 31, 'objective': 'multi:softmax', 'seed': 0}
Tree training time 90.71
Performance evaluation
Compute tight constraints for test set (n_jobs = 1)


Converting dataframe to array: 100%|██████████| 100/100 [00:00<00:00, 1056.01it/s]

Predict tight constraints for test set



100%|██████████| 100/100 [00:12<00:00,  8.05it/s]

Accuracy: 93.00 


## Predict single point

In [10]:
# Predict single point
theta = theta_test.iloc[0]
result_single_point = m.solve(theta)
print(result_single_point)

Converting dataframe to array: 100%|██████████| 1/1 [00:00<00:00, 92.72it/s]

Predict optimal solution



100%|██████████| 1/1 [00:00<00:00,  6.53it/s]


{'x': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 1.]), 'time': 0.002871990203857422, 'strategy': Strategy
  - Tight constraints:
         id: elements
          7: False
         11: [ True False  True  True  True  True  True  True  True False]
         15: [False False False False False False False False False False]
  - Integer variables values:
         id: elements
          0: [0 1 0 0 0 0 0 0 0 1], 'cost': -1.259141227445515, 'infeasibility': 0.0, 'pred_time': 0.0008840560913085938, 'solve_time': 0.001987934112548828}


## Learn directly from points (talk directly to learner)

In [11]:
y = m.y_train
X = m.X_train
learner = XGBoost(n_input=n_features(X),
                  n_classes=len(np.unique(y)),
                  n_best=3,
                  params=params)
# Train learner
learner.train(pandas2array(X), y)

# Predict
X_pred = X.iloc[0]
y_pred = learner.predict(pandas2array(X_pred))  # n_best most likely classes

Converting dataframe to array: 100%|██████████| 1000/1000 [00:00<00:00, 4294.78it/s]

Train XGBoost



/Users/sidereus/miniconda3/envs/python37/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0 
[CV]  batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0, score=0.700, total=   3.1s
[CV] batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.9s remaining:    0.0s


[CV]  batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0, score=0.670, total=   1.6s
[CV] batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0 
[CV]  batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0, score=0.710, total=   1.6s
[CV] batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0 
[CV]  batch_size=32, learning_rate=0.01, max_depth=1, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0, score=0.695, total=   1.9s
[CV] batch_size=32, learning_rate=0.01, max_depth=5, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0 
[CV]  batch_size=32, learning_rate=0.01, max_depth=5, n_epochs=20, n_estimators=50, n_hidden=31, objective=multi:softmax, seed=0, score=0.760,

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.5min finished


Best parameters
{'batch_size': 32, 'learning_rate': 0.1, 'max_depth': 10, 'n_epochs': 20, 'n_estimators': 50, 'n_hidden': 31, 'objective': 'multi:softmax', 'seed': 0}
Tree training time 93.47


Converting dataframe to array: 100%|██████████| 1/1 [00:00<00:00, 1618.17it/s]
